# Road Rage

## Find the ideal speed limit

Drivers do not enter at the same speed at which the exit the road!
Assume that drivers enter the road at the speed limit

In [2]:
# from traffic_lib import *
# pulls things from traffic_lib.py